# Data Anaysis Project part1:

I will follow what I am learning from Google Data Analytics course to complete this project.

* Ask
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so i can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)





## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>

## Process

I noticed that the all the values are string, so I will convert to correct data type as I collect the data. 

In [1]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [2]:
def get_url(city_name, num_post):
    return f'https://{city_name.lower()}.craigslist.org/search/apa?s={num_post}'

In [8]:
# Chilliwack rental housing

response = get(get_url(city_name = 'Abbotsford', num_post = 120))


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
120


In [103]:
%%time
# Find the total number of posts to find the limit of the pagination
city = 'Vancouver'
response = get(get_url(city_name = city, num_post=0))
soup = BeautifulSoup(response.text, 'html.parser')
posts = soup.find_all('li', class_='result-row')
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
print(f"Total number of search result: {results_total}")

# each page has 119 posts so each new page is defined as follows; s=120, s=240, .....
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_cities = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []
post_links = []

for page in pages:
    
    # get request
    response = get(get_url(city_name=city, num_post=page))
    
    sleep(1)
    
    # throw warning for status code that are not 200
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        
    soup = BeautifulSoup(response.text, 'html.parser')
    
    posts = soup.find_all('li', class_='result-row')
    
    
    # Extract data 
    for post in posts:
        
        if post.find('span', class_ = 'result-hood') is not None:
            
            # Posting date
            post_datetime = post.find('time', class_='result-date')['datetime']
            post_date.append(post_datetime)
            
            # Neighbourhoods
            post_city = post.find('span', class_='result-hood').text.strip('( )')
            post_cities.append(post_city)
            
            # title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)
            
            # Post price in integer
            price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
            prices.append(price)
            
            # Post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                num_bedroom.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    num_bedroom.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
        iterations += 1
        #print("Page" + str(iterations) + "Scraped Successfully!")   

Total number of search result: 3000
CPU times: total: 5.19 s
Wall time: 46.2 s


In [104]:
# create DataFrame
df = pd.DataFrame({'Post Datetime': post_date,
                   'Post Title': post_titles,
                   'Post URL': post_links,
                  'Neighborhood': post_cities,
                  'Bedroom': num_bedroom,
                  'SQFT': sqfts,
                  'Price': prices})

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3000 non-null   object 
 1   Post Title     3000 non-null   object 
 2   Post URL       3000 non-null   object 
 3   Neighborhood   3000 non-null   object 
 4   Bedroom        2876 non-null   object 
 5   SQFT           2664 non-null   float64
 6   Price          3000 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 164.2+ KB
None


,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-07 17:33,New Furnished Modern 1 Bedroom 1br suite in Ce...,https://vancouver.craigslist.org/bnc/apa/d/bur...,"Burnaby Near Metrotown EA, and Brentwood BCIT",1,550.0,1790
1,2022-12-07 17:32,Fully Furnished 2 Bedrooms Suite at the Mark -...,https://vancouver.craigslist.org/van/apa/d/van...,1372 Seymour,2,1200.0,8000
2,2022-12-07 17:30,(ORCA_REF#1701-8533) ***PENTHOUSE ALERT! Luxur...,https://vancouver.craigslist.org/van/apa/d/van...,South Marine,4,1886.0,10000
3,2022-12-07 17:29,"Water View, Large 2BR Fully Furnished, office,...",https://vancouver.craigslist.org/van/apa/d/van...,588 Broughton Street,2,950.0,4495
4,2022-12-07 17:29,LARGE 1 BEDROOM,https://vancouver.craigslist.org/van/apa/d/van...,1477 e broadway,1,650.0,1795


In [105]:
df['Bedroom'] == np.nan

0       False
1       False
2       False
3       False
4       False
        ...  
2995    False
2996    False
2997    False
2998    False
2999    False
Name: Bedroom, Length: 3000, dtype: bool

## Process

Check the dataset I created from the Craigslist ad and final adjustment of the dataset to analyze the data. 

First to check the dataset:

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3000 non-null   object 
 1   Post Title     3000 non-null   object 
 2   Post URL       3000 non-null   object 
 3   Neighborhood   3000 non-null   object 
 4   Bedroom        2877 non-null   object 
 5   SQFT           2661 non-null   float64
 6   Price          3000 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 164.2+ KB


I would like to distingish FULLY FURNISHED suite and not furnished suite. I will create another column to show if it isTURE or FALSE. I noticed the title says if it is furnished suite or not (If not on the title, I assume that the suite is not furnished) 

Also, I noticed that the number of bedroom is string, so I will convert to integer. 

In [121]:
# Convert Bedroom number strings to Integer then NaN value to 0 assume 0 bedroom is studio appartment. 
df["Bedroom"] = pd.to_numeric(df["Bedroom"], errors='ignore').astype('Int64')
df["Bedroom"] = df["Bedroom"].fillna(0)

In [107]:
%%time
# Check if word 'furnished' in the title post. 
search_word = 'furnished'
furnished = []
for i in range(len(df)):
    post_title_list = df['Post Title'][i].lower().split(" ")
    if any(word == search_word for word in post_title_list):
        furnished.append(True)
    else:
        furnished.append(False)
df["Furnished?"] = furnished

CPU times: total: 15.6 ms
Wall time: 14 ms


In [123]:
df.head(5)

,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
0,2022-12-07 17:33,New Furnished Modern 1 Bedroom 1br suite in Ce...,https://vancouver.craigslist.org/bnc/apa/d/bur...,"Burnaby Near Metrotown EA, and Brentwood BCIT",1,550.0,1790,True
1,2022-12-07 17:32,Fully Furnished 2 Bedrooms Suite at the Mark -...,https://vancouver.craigslist.org/van/apa/d/van...,1372 Seymour,2,1200.0,8000,True
2,2022-12-07 17:30,(ORCA_REF#1701-8533) ***PENTHOUSE ALERT! Luxur...,https://vancouver.craigslist.org/van/apa/d/van...,South Marine,4,1886.0,10000,False
3,2022-12-07 17:29,"Water View, Large 2BR Fully Furnished, office,...",https://vancouver.craigslist.org/van/apa/d/van...,588 Broughton Street,2,950.0,4495,False
4,2022-12-07 17:29,LARGE 1 BEDROOM,https://vancouver.craigslist.org/van/apa/d/van...,1477 e broadway,1,650.0,1795,False


## Analyze

In [124]:
# Count how many rentals are furnished 
num_furnished_suite = df[df['Furnished?'] == True]["Furnished?"].count()
num_unfurnished_suite = len(df) - num_furnished_suite
furnished_suite_percent = np.round((num_furnished_suite / len(df)) * 100, 2)
print(f"Number of furnished suite is {num_furnished_suite}, and it's {furnished_suite_percent}% of the entire list.")

Number of furnished suite is 448, and it's 14.93% of the entire list.


In [126]:
# Cost of the rent by number of bedroom. 
num_bed = np.sort(df["Bedroom"].unique())
for i in range(len(num_bed)):
    print(f'Number of the suite: {num_bed[i]} on the ads by Number of Bedroom: {len(df[df["Bedroom"] == num_bed[i]])}')

Number of the suite: 0 on the ads by Number of Bedroom: 124
Number of the suite: 1 on the ads by Number of Bedroom: 922
Number of the suite: 2 on the ads by Number of Bedroom: 1228
Number of the suite: 3 on the ads by Number of Bedroom: 407
Number of the suite: 4 on the ads by Number of Bedroom: 192
Number of the suite: 5 on the ads by Number of Bedroom: 90
Number of the suite: 6 on the ads by Number of Bedroom: 24
Number of the suite: 7 on the ads by Number of Bedroom: 7
Number of the suite: 8 on the ads by Number of Bedroom: 6


## Share

## Act